## Scenario: Issuer B cannot update Issuer A's list

In [1]:
from helpers import fixtures, utils
from helpers.utils import tc
import requests
import uuid
import random

### Configure the base URL of the Status List Server

In [2]:
BASE_URL = utils.get_base_url()
print(f"Using base URL: {BASE_URL}")

Using base URL: http://localhost:8000


### Test the Health Check Endpoint (GET /health)

In [3]:
health_endpoint = f"{BASE_URL}/health"
print(f"Testing GET: {health_endpoint}")

response = requests.get(health_endpoint)
tc.assertEqual(response.status_code, 200, "Health check failed")
tc.assertEqual("OK", response.json().get("status"))

print("Health check successful! Server is running.")

Testing GET: http://localhost:8000/health
Health check successful! Server is running.


### Two Issuers register on the Status List Server by publishing credentials

In [4]:
gondwana_digital_pole = fixtures.get_gondwana_digital_pole_issuer()
scott_holdings = fixtures.get_scott_holdings_issuer()

for issuer_data in [gondwana_digital_pole, scott_holdings]:
    print("Proceeding with Issuer:", issuer_data.get('label'))
    
    credentials_endpoint = f"{BASE_URL}/credentials"
    print(f"Testing POST: {credentials_endpoint}")
    
    payload = {
      "issuer": issuer_data.get('label'),
      "public_key": issuer_data.get('keypair').get('public_key'),
      "alg": issuer_data.get('keypair').get('alg')
    }
    
    response = requests.post(credentials_endpoint, json=payload)
    tc.assertEqual(response.status_code, 202, "Failed to publish credentials")
    print("Credentials published successfully!\n")

Proceeding with Issuer: gondwana-digital-pole-nuw01
Testing POST: http://localhost:8000/credentials
Credentials published successfully!

Proceeding with Issuer: scott-holdings-ax3rm
Testing POST: http://localhost:8000/credentials
Credentials published successfully!



### Issuer A publishes a status list

In [5]:
status_publish_endpoint = f"{BASE_URL}/statuslists/publish"
print(f"Testing POST: {status_publish_endpoint}")

issuer_data = gondwana_digital_pole
bearer_token = fixtures.create_bearer_jwt_token(issuer_data)
status_list_id = str(uuid.uuid4())

response = requests.post(status_publish_endpoint, json={
    "list_id": status_list_id,
    "status": [{"index": 1, "status": "VALID"}]
}, headers={"Authorization": f"Bearer {bearer_token}"})

tc.assertEqual(response.status_code, 201, "Failed to publish token statuses")
print(f"Issuer A published status list {status_list_id}")

Testing POST: http://localhost:8000/statuslists/publish
Issuer A published status list f3345f64-e42f-4c8e-8183-ffd751e92fe2


### Issuer B fails to update the status list

In [6]:
status_update_endpoint = f"{BASE_URL}/statuslists/update"
print(f"Testing PATCH: {status_update_endpoint}")

issuer_data = scott_holdings
bearer_token = fixtures.create_bearer_jwt_token(issuer_data)

response = requests.patch(status_update_endpoint, json={
    "list_id": status_list_id,
    "status": [{"index": 1, "status": "INVALID"}]
}, headers={"Authorization": f"Bearer {bearer_token}"})

tc.assertEqual(response.status_code, 403, "Should not be allowed to update another issuer's list")
print(f"Issuer B was prevented from updating status list {status_list_id}")

Testing PATCH: http://localhost:8000/statuslists/update
Issuer B was prevented from updating status list f3345f64-e42f-4c8e-8183-ffd751e92fe2


### Unauthenticated clients are equally prevented from updating the status list

In [7]:
status_update_endpoint = f"{BASE_URL}/statuslists/update"
print(f"Testing PATCH: {status_update_endpoint}")

response = requests.patch(status_update_endpoint, json={
    "list_id": status_list_id,
    "status": [{"index": 1, "status": "INVALID"}]
})

tc.assertEqual(response.status_code, 401, "Authentication should be required on the update endpoint")
print(f"The unauthenticated client was prevented from updating status list {status_list_id}")

Testing PATCH: http://localhost:8000/statuslists/update
The unauthenticated client was prevented from updating status list f3345f64-e42f-4c8e-8183-ffd751e92fe2
